In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Paths for cat and dog directories
TRAIN_IMAGE_DIR = './dogs-vs-cats-identification/train'  # Inside train directory, have separate 'cat' and 'dog' directories

# Use ImageDataGenerator to automatically label data based on folder names
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=0.2  # using 20% of data for validation
)

# Load training and validation data
train_data = datagen.flow_from_directory(
    TRAIN_IMAGE_DIR,
    target_size=(224, 224),
    class_mode='binary',  # Binary since we have two classes
    batch_size=80,
    subset='training'
)

val_data = datagen.flow_from_directory(
    TRAIN_IMAGE_DIR,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=80,
    subset='validation'
)

# Load the pre-trained VGG16 model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-4]:  # Freeze all layers except the last 4
    layer.trainable = False

for layer in base_model.layers[-4:]:  # Set the last 4 layers to trainable
    layer.trainable = True

# Build the custom classification head
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)

# Modify the final layer for binary classification
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)  # Using 2 instead of 120
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

optimizer = tf.keras.optimizers.Adam(lr=1e-5)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Checkpoint
filepath = './models/best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

callbacks = [checkpoint]
batch_size = 80  # Choose a batch size that fits your memory constraints
epochs = 500

history = model.fit(train_data,
                    steps_per_epoch=len(train_data),
                    epochs=epochs,
                    validation_data=val_data,
                    callbacks=callbacks)

test_loss, test_acc = model.evaluate(val_data, verbose=2)
print('\nTest accuracy:', test_acc)
model.save('cat_dog_classifier.h5')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.